# Entrenamiento con Simple Transformers del model BERT

## Paso 1: Cargar datos en Google Colab

## Paso 2: Instalar librerías

In [1]:
!pip install transformers evaluate torch --quiet
!pip install simpletransformers transformers datasets huggingface_hub scikit-learn
!pip install evaluate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.8/330.8 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 140.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 122.2 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=cd47819cdbaa8aa9708ff2c8d776840614cc80b1da2a16a5eff113f94ec70599
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


## Paso 3: Cargar librerías

In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering
import evaluate
import json
import os
import shutil
import requests


from tqdm import tqdm
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
from sklearn.model_selection import train_test_split
from google.colab import files

In [3]:
import logging
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

In [4]:
import torch
print("GPU available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))

GPU available: True
GPU name: NVIDIA A100-SXM4-80GB


## Paso 4: Cargar datos

In [5]:
# URLs of the files
urls = {
    "eval_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/eval_colombia_mexico_dataset.json",
    "train_colombia_mexico_dataset.json": "https://github.com/BlueAutomata/tesis-optimizacion-de-modelos-de-question-answering/raw/refs/heads/master/src/datasets/exploration_datasets/gold/train_colombia_mexico_dataset.json"
}

# Dictionary to store the loaded JSON data
datasets = {}

for filename, url in urls.items():
    # Download the file
    response = requests.get(url)
    if response.status_code == 200:
        # Save locally
        with open(filename, "wb") as f:
            f.write(response.content)
        # Load JSON into Python
        datasets[filename] = response.json()
        print(f"{filename} downloaded and loaded successfully!")
    else:
        print(f"Failed to download {filename}. Status code: {response.status_code}")

eval_colombia_mexico_dataset.json downloaded and loaded successfully!
train_colombia_mexico_dataset.json downloaded and loaded successfully!


In [ ]:
with open("train_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    train_dataset = json.load(f)

print(f"✅ Loaded {len(train_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [ ]:
with open("eval_colombia_mexico_dataset.json", "r", encoding="utf-8") as f:
    eval_dataset = json.load(f)

print(f"✅ Loaded {len(eval_dataset)} records successfully!")

✅ Loaded 1 records successfully!


In [ ]:
train_dataset = train_dataset["data"]

In [ ]:
eval_dataset = eval_dataset["data"]

In [ ]:
# 🔧 1️⃣ Flatten your dataset so each row has 'context' and 'qas'
def flatten_squad(dataset):
    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data

In [ ]:
train_data = flatten_squad(train_dataset)

In [ ]:
eval_data = flatten_squad(eval_dataset)

In [ ]:
print(f"✅ Training samples: {len(train_data)}")
print(f"✅ Eval samples: {len(eval_data)}")

✅ Training samples: 3235
✅ Eval samples: 2845


## Paso 5: Definir hiperparámetros

In [ ]:
model_args = QuestionAnsweringArgs()

# ========================
# ⚙️ Training configuration
# ========================
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 4                # slightly longer training for better convergence
model_args.learning_rate = 3e-5                # standard for BERT fine-tuning
model_args.gradient_accumulation_steps = 4     # effective batch size = 8 × 4 = 32
model_args.warmup_ratio = 0.1
model_args.weight_decay = 0.01                 # regularization to prevent overfitting
model_args.overwrite_output_dir = True

# ========================
# 🧠 Evaluation & Early Stopping
# ========================
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 500
model_args.evaluate_during_training_verbose = True
model_args.save_model_every_epoch = False
model_args.save_eval_checkpoints = True
model_args.save_best_model = True
model_args.metric_for_best_model = "f1"
model_args.greater_is_better = True
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_patience = 2
model_args.early_stopping_consider_epochs = True

# ========================
# 📄 Sequence & Context Settings
# ========================
model_args.max_seq_length = 384       # total sequence length
model_args.doc_stride = 128           # sliding window overlap
model_args.max_query_length = 64      # question length
model_args.max_answer_length = 30

# ========================
# 🧩 Output & Logging
# ========================
model_args.output_dir = "./outputs/"
model_args.best_model_dir = "./outputs/best_model/"
model_args.logging_steps = 100
model_args.manual_seed = 42

# ========================
# ⚡ Resource Handling
# ========================
model_args.use_multiprocessing = False             # safer for notebooks
model_args.fp16 = torch.cuda.is_available()        # enable mixed precision if GPU available
model_args.dataloader_num_workers = 2              # small boost in performance
model_args.save_steps = -1                         # don't save intermediate steps

print("✅ Model arguments configured successfully!")

## Paso 6: Cargar el modelo

In [ ]:
model_original = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

In [ ]:
model = QuestionAnsweringModel(
    model_type="bert",
    model_name="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",  # BETO
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

## Paso 7: Entrenamiento del modelo

In [ ]:
model.train_model(train_data, eval_data=eval_data)

add example index and unique id: 100%|██████████| 14613/14613 [00:00<00:00, 646478.82it/s]


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:697: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 2:   0%|          | 0/8698 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:720: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6263 [00:27<48:28:16, 27.87s/it]

convert squad examples to features:  40%|███▉      | 2501/6263 [00:31<00:35, 106.46it/s]

convert squad examples to features:  80%|███████▉  | 5001/6263 [00:36<00:06, 205.84it/s]

convert squad examples to features: 100%|██████████| 6263/6263 [00:39<00:00, 157.21it/s]


add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 425918.12it/s]


...



convert squad examples to features:   0%|          | 0/6263 [00:00<?, ?it/s]

convert squad examples to features:   0%|          | 1/6263 [00:26<45:51:13, 26.36s/it]

convert squad examples to features:  16%|█▌        | 1001/6263 [00:27<01:40, 52.39it/s]

convert squad examples to features:  24%|██▍       | 1501/6263 [00:30<01:08, 69.81it/s]

convert squad examples to features:  40%|███▉      | 2501/6263 [00:32<00:28, 131.41it/s]

convert squad examples to features:  80%|███████▉  | 5001/6263 [00:37<00:04, 253.21it/s]

convert squad examples to features: 100%|██████████| 6263/6263 [00:39<00:00, 156.85it/s]


add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 358136.12it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]


convert squad examples to features: 100%|██████████| 6263/6263 [00:41<00:00, 152.57it/s]

add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 373924.24it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]

(17396,
 {'global_step': [500,
   1000,
   1500,
   2000,
   2500,
   3000,
   3500,
   4000,
   4500,
   5000,
   5500,
   6000,
   6500,
   7000,
   7500,
   8000,
   8500,
   8698,
   9000,
   9500,
   10000,
   10500,
   11000,
   11500,
   12000,
   12500,
   13000,
   13500,
   14000,
   14500,
   15000,
   15500,
   16000,
   16500,
   17000,
   17396],
  'correct': [3315,
   3640,
   3861,
   3984,
   4072,
   4042,
   4126,
   4125,
   4129,
   4150,
   4180,
   4217,
   4185,
   4232,
   4220,
   4263,
   4238,
   4253,
   4278,
   4275,
   4261,
   4257,
   4228,
   4263,
   4263,
   4284,
   4279,
   4292,
   4288,
   4279,
   4291,
   4286,
   4289,
   4294,
   4295,
   4293],
  'similar': [2656,
   2303,
   2098,
   1890,
   1868,
   1934,
   1804,
   1815,
   1777,
   1795,
   1781,
   1700,
   1805,
   1719,
   1717,
   1691,
   1722,
   1710,
   1673,
   1690,
   1691,
   1700,
   1691,
   1698,
   1679,
   1670,
   1682,
   1665,
   1672,
   1689,
   1669,
   1674,
  

## Paso 8: Evaluación de los resultados

In [ ]:
result_original, texts_original = model_original.eval_model(eval_data)
print("📊 Evaluation results:")
print(result_original)

add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 364638.55it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]

📊 Evaluation results:
{'correct': 3075, 'similar': 2614, 'incorrect': 574, 'eval_loss': -6.884965852649007}


In [ ]:
result, texts = model.eval_model(eval_data)
print("📊 Evaluation results:")
print(result)

add example index and unique id: 100%|██████████| 6263/6263 [00:00<00:00, 379910.71it/s]


Running Evaluation:   0%|          | 0/3775 [00:00<?, ?it/s]

📊 Evaluation results:
{'correct': 4293, 'similar': 1664, 'incorrect': 306, 'eval_loss': -9.040801945364239}


In [ ]:
correct = result_original['correct']
similar = result_original['similar']
incorrect = result_original['incorrect']
total = correct + similar + incorrect

# 1️⃣ Exact Match Accuracy
exact_match = correct / total

# 2️⃣ Weighted Accuracy (partial credit for 'similar')
weighted_accuracy = (correct + 0.5 * similar) / total

# 3️⃣ F1 Score approximation
TP = correct + 0.5 * similar
FN = 0.5 * similar + incorrect
# Assuming FP = 0 (as Simple Transformers counts predictions, not negatives)
precision = TP / TP
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results
print(f"Exact Match (EM): {exact_match:.4f} → {exact_match*100:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.4f} → {weighted_accuracy*100:.2f}%")
print(f"F1 Score: {f1_score:.4f} → {f1_score*100:.2f}%")

Exact Match (EM): 0.4910 → 49.10%
Weighted Accuracy: 0.6997 → 69.97%
F1 Score: 0.8233 → 82.33%


In [ ]:
correct = result['correct']
similar = result['similar']
incorrect = result['incorrect']
total = correct + similar + incorrect

# 1️⃣ Exact Match Accuracy
exact_match = correct / total

# 2️⃣ Weighted Accuracy (partial credit for 'similar')
weighted_accuracy = (correct + 0.5 * similar) / total

# 3️⃣ F1 Score approximation
TP = correct + 0.5 * similar
FN = 0.5 * similar + incorrect
# Assuming FP = 0 (as Simple Transformers counts predictions, not negatives)
precision = TP / TP
recall = TP / (TP + FN)
f1_score = 2 * (precision * recall) / (precision + recall)

# Print results
print(f"Exact Match (EM): {exact_match:.4f} → {exact_match*100:.2f}%")
print(f"Weighted Accuracy: {weighted_accuracy:.4f} → {weighted_accuracy*100:.2f}%")
print(f"F1 Score: {f1_score:.4f} → {f1_score*100:.2f}%")

Exact Match (EM): 0.6855 → 68.55%
Weighted Accuracy: 0.8183 → 81.83%
F1 Score: 0.9001 → 90.01%


## Paso 9: Guardar los resultados

In [ ]:
# Folder to save
local_path = "./QA_model_bert"
os.makedirs(local_path, exist_ok=True)

# Save the Hugging Face model & tokenizer directly
model.model.save_pretrained(local_path)       # Saves weights + config
model.tokenizer.save_pretrained(local_path)   # Saves vocab + tokenizer config

# Check files
!ls -l ./QA_model

ls: cannot access './QA_model': No such file or directory


In [ ]:
shutil.make_archive("QA_model_bert", 'zip', local_path)
print("✅ Zipped model")
!ls -lh QA_model.zip

✅ Zipped model
ls: cannot access 'QA_model.zip': No such file or directory


In [ ]:
files.download("QA_model_bert.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login

# This will open a prompt for your Hugging Face token
login()

In [ ]:
from huggingface_hub import whoami
print(whoami())


{'type': 'user', 'id': '6682cc422b6af3f60a185123', 'name': 'BlueAutomata', 'fullname': 'Guillermo Luigui Ubaldo Nieto Angarita', 'email': 'guillermo.luigui.nieto@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/b6cf26e7fac6e034fc5f2b2b87f9ff70.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'write_token', 'role': 'write', 'createdAt': '2025-10-26T19:35:35.613Z'}}}


In [ ]:

from huggingface_hub import login, create_repo
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

# 1️⃣ Define your paths and repo name
model_dir = "./outputs"
repo_id = "BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico"

# 2️⃣ Create the repo (won’t fail if it already exists)
create_repo(repo_id, private=False, exist_ok=True)

# 3️⃣ Load your SimpleTransformers model as a standard HF model
hf_model = AutoModelForQuestionAnswering.from_pretrained(model_dir)
hf_tokenizer = AutoTokenizer.from_pretrained(model_dir)

# 4️⃣ Push to the Hugging Face Hub
hf_model.push_to_hub(
    repo_id,
    description="BERT-base Spanish WWM cased model fine-tuned for extractive QA on news articles from Colombia and Mexico.",
    tags=["spanish", "qa", "news", "colombia", "mexico", "bert-base", "wwm", "cased"]
)

hf_tokenizer.push_to_hub(repo_id)

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...0eaw9mg/model.safetensors:   0%|          |  558kB /  437MB            

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico/commit/1ed334653334e746169668dd1d0d68f0e0d64070', commit_message='Upload tokenizer', commit_description='', oid='1ed334653334e746169668dd1d0d68f0e0d64070', pr_url=None, repo_url=RepoUrl('https://huggingface.co/BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico', endpoint='https://huggingface.co', repo_type='model', repo_id='BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico'), pr_revision=None, pr_num=None)

In [ ]:

shutil.move("QA_model_bert.zip", "/content/drive/MyDrive/Tahesis_QA_Optimization/Model")

'/content/drive/MyDrive/Thesis_QA_Optimization/Model/QA_model_bert.zip'

In [ ]:
# Path to the folder containing the saved model
model_path = "./QA_model_bert"  # change if different

# Reload the model
my_model = QuestionAnsweringModel(
    "bert",
    model_path,
    use_cuda=True  # set to False if no GPU
)

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

In [ ]:
# Context & question
context = "Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas."
question = "¿Quién fue hallado muerto en Tamaulipas?"

# Prepare input in SimpleTransformers format
to_predict = [
    {
        "context": context,
        "qas": [
            {
                "id": "0",
                "question": question,
                "answers": [{"text": " ", "answer_start": 0}],
                "is_impossible": False
            }
        ]
    }
]

# Run prediction
answers = my_model.predict(to_predict)
print(answers)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11397.57it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '0', 'answer': ['Abraham Jeremías Pérez Ramírez,', 'El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Ramírez', 'capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez', 'Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto en Tamaulipas.', 'Abraham', 'Ciudad de México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Pérez Ramírez', 'Abraham Jeremías', 'la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado muerto', 'de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue', 'de Marina, Abraham Jeremías Pérez Ramírez,', 'México. El capitán de la Secretaría de Marina, Abraham Jeremías Pérez Ramírez,', 'Abraham Jeremías Pérez Ramírez, fue hallado']}], [{'id': '0', 'probability': 

/usr/local/lib/python3.12/dist-packages/simpletransformers/question_answering/question_answering_model.py:1358: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [ ]:
# Load your fine-tuned model from the Hub
qa = pipeline(
    "question-answering",
    model="BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    tokenizer="BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico"
)

# Provide Spanish context
contexto = """
El presidente Gustavo Petro anunció nuevas medidas para impulsar el uso de energías renovables en Colombia,
especialmente en la región del Caribe, donde los proyectos solares y eólicos han ganado protagonismo.
El objetivo del gobierno es reducir las emisiones de carbono en un 30% para el año 2030.
"""

# Ask questions in Spanish
preguntas = [
    "¿Quién anunció nuevas medidas para energías renovables?",
    "¿En qué región se impulsarán los proyectos solares y eólicos?",
    "¿Cuál es el objetivo del gobierno para 2030?"
]

# Evaluate each question
for pregunta in preguntas:
    respuesta = qa(question=pregunta, context=contexto)
    print(f"❓ {pregunta}\n💬 {respuesta['answer']}\n")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/437M [00:00<?, ?B/s]

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Device set to use cuda:0


❓ ¿Quién anunció nuevas medidas para energías renovables?
💬 Gustavo Petro

❓ ¿En qué región se impulsarán los proyectos solares y eólicos?
💬 Caribe

❓ ¿Cuál es el objetivo del gobierno para 2030?
💬 reducir las emisiones de carbono en un 30%



In [ ]:
def flatten_squad(dataset):
    # If the dataset is a dict with "data", extract it
    if isinstance(dataset, dict) and "data" in dataset:
        dataset = dataset["data"]

    new_data = []
    for article in dataset:
        for para in article["paragraphs"]:
            new_data.append({
                "title": article.get("title", ""),
                "context": para["context"],
                "qas": para["qas"]
            })
    return new_data


In [ ]:
flat_eval = flatten_squad(eval_dataset)

In [ ]:
from transformers import pipeline
import evaluate

qa_pipeline = pipeline(
    "question-answering",
    model="BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico",
    tokenizer="BlueAutomata/bert-base-spanish-wwm-cased-news-qa-colombia-mexico"
)

metric = evaluate.load("squad")

The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

In [ ]:
from transformers import pipeline
import evaluate

qa_pipeline_original = pipeline(
    "question-answering",
    model="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es",
    tokenizer="mrm8488/bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"
)


The following layers were not sharded: bert.encoder.layer.*.attention.output.LayerNorm.bias, bert.encoder.layer.*.attention.self.value.weight, qa_outputs.weight, bert.encoder.layer.*.output.dense.weight, bert.encoder.layer.*.attention.self.value.bias, qa_outputs.bias, bert.encoder.layer.*.attention.output.dense.weight, bert.encoder.layer.*.output.LayerNorm.weight, bert.encoder.layer.*.output.LayerNorm.bias, bert.encoder.layer.*.attention.output.dense.bias, bert.encoder.layer.*.intermediate.dense.weight, bert.encoder.layer.*.output.dense.bias, bert.embeddings.word_embeddings.weight, bert.encoder.layer.*.intermediate.dense.bias, bert.encoder.layer.*.attention.self.key.weight, bert.encoder.layer.*.attention.self.query.weight, bert.embeddings.position_embeddings.weight, bert.embeddings.token_type_embeddings.weight, bert.encoder.layer.*.attention.self.query.bias, bert.embeddings.LayerNorm.bias, bert.encoder.layer.*.attention.output.LayerNorm.weight, bert.encoder.layer.*.attention.self.key.b

In [ ]:
predictions_original = []
references_original = []

for ex in flat_eval:
    context = ex["context"]
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            continue  # Skip unanswerable questions

        # Run QA prediction
        pred = qa_pipeline_original(question=qa["question"], context=context)

        # Collect prediction and reference
        predictions_original.append({
            "id": qa["id"],
            "prediction_text": pred["answer"]
        })

        references_original.append({
            "id": qa["id"],
            "answers": {
                "text": [a["text"] for a in qa["answers"]],
                "answer_start": [a["answer_start"] for a in qa["answers"]]
            }
        })


In [ ]:
predictions = []
references = []

for ex in flat_eval:
    context = ex["context"]
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            continue  # Skip unanswerable questions

        # Run QA prediction
        pred = qa_pipeline(question=qa["question"], context=context)

        # Collect prediction and reference
        predictions.append({
            "id": qa["id"],
            "prediction_text": pred["answer"]
        })

        references.append({
            "id": qa["id"],
            "answers": {
                "text": [a["text"] for a in qa["answers"]],
                "answer_start": [a["answer_start"] for a in qa["answers"]]
            }
        })


In [ ]:
results = metric.compute(predictions=predictions_original, references=references_original)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 60.14
F1 Score: 77.00


In [ ]:
results = metric.compute(predictions=predictions, references=references)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact_match']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 75.21
F1 Score: 85.09


In [ ]:
import evaluate
metric = evaluate.load("squad_v2")

In [ ]:
from tqdm import tqdm

predictions_original = []
references_original = []

for ex in tqdm(flat_eval):
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            # Questions that have no valid answer in the text
            predictions_original.append({
                "id": qa["id"],
                "prediction_text": "",
                "no_answer_probability": 1.0   # fully confident it's unanswerable
            })
            references_original.append({
                "id": qa["id"],
                "answers": {"text": [], "answer_start": []}
            })
        else:
            # Normal (answerable) questions
            pred = qa_pipeline_original(question=qa["question"], context=ex["context"])

            predictions_original.append({
                "id": qa["id"],
                "prediction_text": pred["answer"],
                # Use model confidence inversely as no-answer probability
                "no_answer_probability": 1.0 - pred.get("score", 0.0)
            })

            references_original.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

100%|██████████| 2845/2845 [03:10<00:00, 14.90it/s]


In [ ]:
from tqdm import tqdm

predictions = []
references = []

for ex in tqdm(flat_eval):
    for qa in ex["qas"]:
        if qa["is_impossible"]:
            # Questions that have no valid answer in the text
            predictions.append({
                "id": qa["id"],
                "prediction_text": "",
                "no_answer_probability": 1.0   # fully confident it's unanswerable
            })
            references.append({
                "id": qa["id"],
                "answers": {"text": [], "answer_start": []}
            })
        else:
            # Normal (answerable) questions
            pred = qa_pipeline(question=qa["question"], context=ex["context"])

            predictions.append({
                "id": qa["id"],
                "prediction_text": pred["answer"],
                # Use model confidence inversely as no-answer probability
                "no_answer_probability": 1.0 - pred.get("score", 0.0)
            })

            references.append({
                "id": qa["id"],
                "answers": {
                    "text": [a["text"] for a in qa["answers"]],
                    "answer_start": [a["answer_start"] for a in qa["answers"]]
                }
            })

100%|██████████| 2845/2845 [02:58<00:00, 15.96it/s]


In [ ]:
predictions_original

[{'id': 'qa-15740', 'prediction_text': 'Illinois'},
 {'id': 'qa-15744', 'prediction_text': 'Salón México'},
 {'id': 'qa-15745', 'prediction_text': ''},
 {'id': 'qa-15746', 'prediction_text': ''},
 {'id': 'qa-15859', 'prediction_text': ''},
 {'id': 'qa-15856',
  'prediction_text': 'Instituto Nacional de Bellas Artes y Literatura'},
 {'id': 'qa-15901',
  'prediction_text': 'investigación participativa y trabajo comunitario'},
 {'id': 'qa-15902', 'prediction_text': ''},
 {'id': 'qa-15431', 'prediction_text': ''},
 {'id': 'qa-15432', 'prediction_text': ''},
 {'id': 'qa-15426', 'prediction_text': 'Lucina Jiménez'},
 {'id': 'qa-15037', 'prediction_text': ''},
 {'id': 'qa-19941', 'prediction_text': 'Claudia Sheinbaum Pardo'},
 {'id': 'qa-19940', 'prediction_text': '9 por ciento'},
 {'id': 'qa-18801', 'prediction_text': ''},
 {'id': 'qa-17754', 'prediction_text': 'Organización de Salud Mundial'},
 {'id': 'qa-17755', 'prediction_text': 'líderes árabes'},
 {'id': 'qa-9297', 'prediction_text': 'G

In [ ]:
results = metric.compute(predictions=predictions_original, references=references_original)

print("📊 Evaluation results:")
print(f"Exact Match: {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 72.17
F1 Score: 83.94


In [ ]:
results = metric.compute(predictions=predictions, references=references)
print("📊 Evaluation results:")
print(f"Exact Match: {results['exact']:.2f}")
print(f"F1 Score: {results['f1']:.2f}")

📊 Evaluation results:
Exact Match: 82.69
F1 Score: 89.59


In [ ]:
!mv outputs outputs_bert
!zip -r outputs_bert.zip outputs_bert

mv: cannot stat 'outputs': No such file or directory
  adding: outputs_bert/ (stored 0%)
  adding: outputs_bert/config.json (deflated 49%)
  adding: outputs_bert/model.safetensors (deflated 7%)
  adding: outputs_bert/predictions_test.json (deflated 70%)
  adding: outputs_bert/checkpoint-17396-epoch-2/ (stored 0%)
  adding: outputs_bert/checkpoint-17396-epoch-2/eval_results.txt (deflated 10%)
  adding: outputs_bert/model_args.json (deflated 61%)
  adding: outputs_bert/training_progress_scores.csv (deflated 51%)
  adding: outputs_bert/null_odds_test.json (deflated 76%)
  adding: outputs_bert/nbest_predictions_test.json (deflated 87%)
  adding: outputs_bert/special_tokens_map.json (deflated 42%)
  adding: outputs_bert/vocab.txt (deflated 54%)
  adding: outputs_bert/tokenizer_config.json (deflated 74%)
  adding: outputs_bert/best_model/ (stored 0%)
  adding: outputs_bert/best_model/config.json (deflated 49%)
  adding: outputs_bert/best_model/model.safetensors (deflated 7%)
  adding: output

In [ ]:
shutil.move("outputs_bert.zip", "/content/drive/MyDrive/Thesis_QA_Optimization/Model")

Error: Destination path '/content/drive/MyDrive/Thesis_QA_Optimization/Model/outputs_bert.zip' already exists

In [ ]:
import os
import shutil

# Define the full path to the destination directory
destination_dir = '/content/drive/MyDrive/Thesis_QA_Optimization/Model'
source_file = 'outputs_bert.zip' # This is the file you want to move

# 1. Check if the directory exists and create it if it doesn't
# The `exist_ok=True` argument prevents an error if the directory already exists.
# The `os.makedirs` function creates all intermediate-level directories needed.
os.makedirs(destination_dir, exist_ok=True)

# 2. Now you can safely move the file
try:
    shutil.move(source_file, destination_dir)
    print(f"Successfully moved {source_file} to {destination_dir}")
except FileNotFoundError as e:
    # This might catch a different FileNotFoundError if the source file doesn't exist,
    # but the primary directory issue should be resolved by os.makedirs.
    print(f"Error moving file: {e}")

Successfully moved outputs_bert.zip to /content/drive/MyDrive/Tahesis_QA_Optimization/Model
